## Get Tweets

In [1]:
from twitterscraper import query_tweets
from datetime import timedelta, date
import pandas as pd
import csv
%matplotlib inline

In [2]:
def twitter_query(query, start_date, end_date,limit_per_day=60):
    
    query = query_tweets(query=query, 
                     limit=limit_per_day, 
                     begindate=start_date, 
                     enddate=end_date, 
                     poolsize=20, 
                     lang='en')
    
    return query

In [3]:
def create_list(query):    
    tweets = []
    for tweet in query:
        tweets.append({'date': tweet.timestamp, 'text': tweet.text, 
                   'fullname': tweet.fullname, 'id': tweet.id, 
                   'likes': tweet.likes, 'replies': tweet.replies,
                   'retweets': tweet.retweets, 'url': tweet.url,
                   'user': tweet.user})
    
    return tweets

In [4]:
def add_to_list(query):
    for tweet in query:
        tweets.append({'date': tweet.timestamp, 'text': tweet.text, 
                   'fullname': tweet.fullname, 'id': tweet.id, 
                   'likes': tweet.likes, 'replies': tweet.replies,
                   'retweets': tweet.retweets, 'url': tweet.url,
                   'user': tweet.user})
    
    return tweets

### Query

In [30]:
query = '#schumershutdown'
start_date = date(2018, 1, 16)
end_date = date(2018, 1, 25)
filename = ''

query = twitter_query(query, start_date, end_date)

OSError: [Errno 24] Too many open files

In [6]:

tweets = create_list(query)

In [24]:
if len(tweets) > 0:
    tweets = add_to_list(query)

## Pre-process Tweets

In [28]:
def pre_process(tweets):
    df = pd.DataFrame(tweets)
    print("Initial dataframe length: ", len(df))
    
    # add timestamp column
    df['month'] = df['date'].apply(lambda x : date(x.year, x.month, 1))
    
    # drop NAs
    df = df.dropna()
    # drop duplicates
    df = df.drop_duplicates()
    
    df['text'] = df['text'].apply(str)
    
    print("# of Unique ID's: ", len(df.id.unique()))
    print("Final dataframe length: ", len(df.id.unique()))
    return df

In [29]:
df = pre_process(tweets)

Initial dataframe length:  901
# of Unique ID's:  323
Final dataframe length:  323


In [15]:
# drop values outside of queried range
#df = df[df['date'] >= start_date]
#df = df[df['date'] <= end_date]

In [27]:
len(df)

325

### Check that tweets look normal, ready for export

In [31]:
df.sort_values('date').head()

,date,fullname,id,likes,replies,retweets,text,url,user,month
90,2018-01-15 21:48:35,Marcia Wood,953021120021540864,0,0,0,#Flashback: That Time Schumer Said It Was Insa...,/littletboca/status/953021120021540864,littletboca,2018-01-01
89,2018-01-15 21:49:58,Melanie,953021468102873088,2,0,1,Flashback: That Time Schumer Said It Was Insan...,/mefbama/status/953021468102873088,mefbama,2018-01-01
88,2018-01-15 21:55:24,Eagle 1,953022832291778560,0,0,0,"Schumer, Pelosi refuse to meet with Trump as s...",/johncassity1/status/953022832291778560,johncassity1,2018-01-01
87,2018-01-15 22:13:47,HEDGEaccordingly 📈,953027457896079360,0,0,1,"The top two Democrats Schumer, Pelosi cancel m...",/HedgeBz/status/953027457896079360,HedgeBz,2018-01-01
86,2018-01-15 22:14:08,GJMHH,953027549533298690,0,0,0,Flashback: That Time Schumer Said It Was Insan...,/my87srx250/status/953027549533298690,my87srx250,2018-01-01


In [33]:
df.groupby('date')['text'].count().cumsum().plot()

TypeError: First argument must be a path or file object reading bytes

In [34]:
df.to_csv('twitter_data/'+filename)